In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [134]:
## Initial Imports

%matplotlib inline

import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import random
from tqdm import tqdm_notebook as tqdm
from sklearn.model_selection import LeaveOneOut, GridSearchCV, KFold, train_test_split
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report, matthews_corrcoef, roc_auc_score
from statistics import mean
from imblearn.over_sampling import SMOTE, BorderlineSMOTE, ADASYN
from imblearn.combine import SMOTETomek
import imblearn
from collections import Counter
random.seed(10)

In [135]:
pdh = pd.read_csv(r'C:\Users\User\Videos\PD paper smote oversampling/gender_class_FH_corrected_Copy.csv')
pdh = pd.read_csv(r'C:\Users\User\Videos\PD paper smote oversampling/tqwt_mfccs_concat_FH.csv')
print(pdh.head())
#print('The shape of the matrix is :', pdh.shape)
print(list(pdh.columns))

   tqwt_maxValue_dec_14  tqwt_entropy_log_dec_33  tqwt_meanValue_dec_17  \
0              0.105750               -4540.0830           9.600000e-20   
1              0.121580               -4466.8949          -1.180000e-19   
2              0.081818               -4455.2826          -7.420000e-20   
3              0.567740               -2779.8086          -1.070000e-19   
4              0.567490               -2910.9996           1.310000e-19   

   tqwt_meanValue_dec_27  tqwt_entropy_log_dec_35  tqwt_entropy_log_dec_10  \
0           1.840000e-18               -3190.1752             -281439.2690   
1          -4.270000e-18               -3106.4317             -285733.2255   
2          -2.120000e-18               -3082.5691             -305145.0185   
3           5.940000e-19               -1965.3808             -287896.7686   
4           1.280000e-19               -2102.3852             -279129.6866   

   tqwt_meanValue_dec_3  tqwt_entropy_log_dec_28  tqwt_meanValue_dec_6  \
0     

In [136]:
df=pdh
train_df=df.values
x_train=train_df[:,:-1]
y_train=train_df[:,-1]
print(Counter(y_train))
print(x_train.shape)
print(y_train.shape)
print(len(y_train))

Counter({1.0: 564, 0.0: 192})
(756, 118)
(756,)
756


In [137]:
pdg_gen=pdg.values
pdg_gen[5,-2]

NameError: name 'pdg' is not defined

In [138]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train= sc.fit_transform(x_train)
#newly added

kfold=KFold(252, shuffle=False)#random_state=10

In [140]:
def svm_model(X,y, kfold):
    params={'C':[0.1, 1, 10, 50,], 'gamma':[ 0.01, 0.001, 0.0001], 'kernel':['rbf']}
    over_samp=ADASYN(sampling_strategy='not minority', n_neighbors=4, )#
    #over_samp2=imblearn.combine.SMOTEENN( sampling_strategy='not minority', random_state=None, smote=None, enn=None, n_jobs=None)
    #trainx, trainy=over_samp.fit_resample(trainx, trainy)
    X, y=over_samp.fit_resample(trainx, trainy)
    grid=GridSearchCV(svm.SVC(probability=True, random_state=10),params,cv=kfold, refit=True, verbose=1)
    grid.fit(X,y)
    
    return grid

In [141]:
grid=svm_model(x_train, y_train,kfold)

Fitting 252 folds for each of 12 candidates, totalling 3024 fits


In [142]:
from statistics import mean
print(grid.best_params_)
print(grid.best_score_)
print(grid.cv_results_)

{'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}
0.8373015873015873
{'mean_fit_time': array([0.56527566, 0.45723881, 0.34711657, 0.49540318, 0.36876898,
       0.33450726, 0.56311017, 0.21006549, 0.17032379, 0.29562686,
       0.2043892 , 0.17121329]), 'std_fit_time': array([0.24927125, 0.14302273, 0.07418871, 0.10327712, 0.08299393,
       0.03457533, 0.3327467 , 0.10526033, 0.02289204, 0.04511637,
       0.02980544, 0.03368419]), 'mean_score_time': array([0.00221924, 0.00218715, 0.00177964, 0.00210356, 0.00168396,
       0.00155667, 0.00178824, 0.00070206, 0.0008968 , 0.00093618,
       0.00084113, 0.00084973]), 'std_score_time': array([0.0018474 , 0.00182181, 0.00152774, 0.00208718, 0.00124251,
       0.00119721, 0.00233798, 0.00143369, 0.00288067, 0.0023806 ,
       0.00263095, 0.00175743]), 'param_C': masked_array(data=[0.1, 0.1, 0.1, 1, 1, 1, 10, 10, 10, 50, 50, 50],
             mask=[False, False, False, False, False, False, False, False,
                   False, False, False, False

In [64]:
def oversampling_smote(trainx,trainy, xs=2):
    
    # uses concept of smote for oversampling
    
    attrs=trainx.shape[1] # getting number of attributes
    synthetic_sample=np.empty((3*(xs-1), attrs))
    overx=trainx[0:3,:]
    overy=[]
    
    for i in range(0,len(trainx),3):
        if(trainy[i]==0):
            gap1=trainx[i]-trainx[i+1]
            gap2=trainx[i]-trainx[i+2]
            gap3=trainx[i+1]-trainx[i+2]
            
            for j in range(synthetic_sample.shape[0],3):
                synthetic_sample[j]=trainx[i]+random.uniform(0,1)*gap1#trainx[i]+random.uniform(0,1)*gap1
                synthetic_sample[j+1]=trainx[i+2]+random.uniform(0,1)*gap2
                synthetic_sample[j+2]=train[i+1]+random.uniform(0,1)*gap3
            
            overx=np.vstack((overx,trainx[i:i+3,:]))
            overx=np.vstack((overx, synthetic_sample))
            
            for j in trainy[i:(i+3)]:
                overy.append(j)
            
            for l in range(synthetic_sample.shape[0]):
                overy.append(trainy[i])
            
        else:
            overx=np.vstack((overx,trainx[i:i+3,:]))
            for j in trainy[i:i+3]:
                overy.append(j)
    overx=overx[3:,:]#cropping of the 1st 3 because it is reapeated
    overy=np.array(overy)
    print(overx.shape)
    print(overy.shape)
    print(Counter(overy))
    return overx,overy

In [162]:
wrong_pred_indices=[]
d={}
svm_estimator=svm.SVC(C= 12, gamma= 0.001, kernel= 'poly',probability=True, random_state=10)
i=0

correct_ones_=[]#collects correct pd postive predictions
correct_zeros_=[]#collects correct pd negative predictions

wrong_ones_=[]#collect wrong pd positive predictions
wrong_zeros_=[]#collect wrong pd negative predictions

tp=[]# true positive
tn=[]# true negative
fp=[]# false positive
fn=[]# false negative

tridx=[]
tstidx=[]
acc=[]
mcc=[]
f1=[]
tsts=[]

for trids, tstids in kfold.split(x_train):
    
    trainx, testx= x_train[trids,:], x_train[tstids,:]
    trainy, testy= y_train[trids], y_train[tstids]
    
    #perform oversampling
    #over_samp=ADASYN(sampling_strategy='all')#BorderlineSMOTE()
    #trainx, trainy=over_samp.fit_resample(trainx, trainy)
    #print(Counter(trainy))

    ''''for i in range(0, len(trainx),3):
        #print(type(x_train[i:i+3,:]))
        if trainy[i]==0:
            a=np.vstack((a,trainx[i:i+3,:]))
            a=np.vstack((a,trainx[i:i+3,:]))
            a=np.vstack((a,trainx[i:i+3,:]))
    
            for j in trainy[i:(i+3)]:
                sampy.append(j)
                sampy.append(j)
                sampy.append(j)
        else:
            a=np.vstack((a,trainx[i:i+3,:]))
            for j in trainy[i:(i+3)]:
                sampy.append(j)
    a=a[3:,:]
    print(a.shape), print(np.array(sampy).shape)
    
    trainx=a
    trainy=np.array(sampy)'''
    

    trainx, trainy=oversampling_smote(trainx,trainy, xs=2)
    over_samp=SMOTETomek(sampling_strategy='not majority')#ADASYN(sampling_strategy='not minority', n_neighbors=4, )#
    #over_samp2=imblearn.combine.SMOTEENN( sampling_strategy='not minority', random_state=None, smote=None, enn=None, n_jobs=None)
    #trainx, trainy=over_samp.fit_resample(trainx, trainy)
    trainx, trainy=over_samp.fit_resample(trainx, trainy)
    
    svm_estimator=svm.SVC(C= 10, gamma= 0.001, tol=1e-3, kernel= 'rbf',probability=True, random_state=10)
    
    svm_estimator.fit(trainx, trainy)
    
    preds=svm_estimator.predict(x_train[tstids,:])
    
    acc.append(accuracy_score(testy,svm_estimator.predict(testx)))
    #mcc.append(matthews_corrcoef(testy, svm_estimator.predict(testx)))
    f1.append(f1_score(testy, svm_estimator.predict(testx)))
    

    i=i+1
    print(i)
#print('acc:', mean(acc))
#print('f1:', mean(f1))
#print('mcc:', mean(mcc))


(945, 118)
(945,)
Counter({1.0: 561, 0.0: 384})
1
(945, 118)
(945,)
Counter({1.0: 561, 0.0: 384})
2
(945, 118)
(945,)
Counter({1.0: 561, 0.0: 384})
3
(945, 118)
(945,)
Counter({1.0: 561, 0.0: 384})
4
(945, 118)
(945,)
Counter({1.0: 561, 0.0: 384})
5
(945, 118)
(945,)
Counter({1.0: 561, 0.0: 384})
6
(945, 118)
(945,)
Counter({1.0: 561, 0.0: 384})
7
(945, 118)
(945,)
Counter({1.0: 561, 0.0: 384})
8
(945, 118)
(945,)
Counter({1.0: 561, 0.0: 384})
9
(945, 118)
(945,)
Counter({1.0: 561, 0.0: 384})
10
(945, 118)
(945,)
Counter({1.0: 561, 0.0: 384})
11
(942, 118)
(942,)
Counter({1.0: 564, 0.0: 378})
12
(945, 118)
(945,)
Counter({1.0: 561, 0.0: 384})
13
(945, 118)
(945,)
Counter({1.0: 561, 0.0: 384})
14
(942, 118)
(942,)
Counter({1.0: 564, 0.0: 378})
15
(945, 118)
(945,)
Counter({1.0: 561, 0.0: 384})
16
(945, 118)
(945,)
Counter({1.0: 561, 0.0: 384})
17
(945, 118)
(945,)
Counter({1.0: 561, 0.0: 384})
18
(945, 118)
(945,)
Counter({1.0: 561, 0.0: 384})
19
(945, 118)
(945,)
Counter({1.0: 561, 0.0

F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.


30
(945, 118)
(945,)
Counter({1.0: 561, 0.0: 384})
31
(945, 118)
(945,)
Counter({1.0: 561, 0.0: 384})
32
(945, 118)
(945,)
Counter({1.0: 561, 0.0: 384})
33
(942, 118)
(942,)
Counter({1.0: 564, 0.0: 378})


F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.


34
(942, 118)
(942,)
Counter({1.0: 564, 0.0: 378})
35
(945, 118)
(945,)
Counter({1.0: 561, 0.0: 384})
36
(945, 118)
(945,)
Counter({1.0: 561, 0.0: 384})
37
(945, 118)
(945,)
Counter({1.0: 561, 0.0: 384})
38
(945, 118)
(945,)
Counter({1.0: 561, 0.0: 384})
39
(945, 118)
(945,)
Counter({1.0: 561, 0.0: 384})
40
(945, 118)
(945,)
Counter({1.0: 561, 0.0: 384})
41
(942, 118)
(942,)
Counter({1.0: 564, 0.0: 378})


...

245
(942, 118)
(942,)
Counter({1.0: 564, 0.0: 378})
246
(942, 118)
(942,)
Counter({1.0: 564, 0.0: 378})


F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.


247
(945, 118)
(945,)
Counter({1.0: 561, 0.0: 384})
248
(945, 118)
(945,)
Counter({1.0: 561, 0.0: 384})
249
(945, 118)
(945,)
Counter({1.0: 561, 0.0: 384})
250
(942, 118)
(942,)
Counter({1.0: 564, 0.0: 378})


F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.


251
(942, 118)
(942,)
Counter({1.0: 564, 0.0: 378})
252


In [163]:
j=0
tpids=[]
tpc=0
tnids=[]
tnc=0
fpids=[]
fpc=0
fnids=[]
fnc=0

#performing majority voting
# if accurately predicted over 0.66 for 3 that means we predicted correctly for the whole sample

for t in range(0,len(y_train),3):
    print('t:',t," original:", y_train[t],' patient no: ',j, 'correct prediction rate:',acc[j] )

    
    if y_train[t]==1 and acc[j]>0.5:
        tpc=tpc+1
        tpids.append(t)
    if y_train[t]==1 and acc[j]<0.5:
        fnc=fnc+1
        fnids.append(t)
    if y_train[t]==0 and acc[j]>0.5:
        tnc=tnc+1
        tnids.append(t)
    if y_train[t]==0 and acc[j]<0.5:
        fpc=fpc+1
        fpids.append(t)
    
    j=j+1

print("true positives: ", tpc)
print("true negatives: ", tnc)
print("fales positives: ", fpc)
print("false negatives: ", fnc)

t: 0  original: 1.0  patient no:  0 correct prediction rate: 1.0
t: 3  original: 1.0  patient no:  1 correct prediction rate: 1.0
t: 6  original: 1.0  patient no:  2 correct prediction rate: 0.3333333333333333
t: 9  original: 1.0  patient no:  3 correct prediction rate: 1.0
t: 12  original: 1.0  patient no:  4 correct prediction rate: 1.0
t: 15  original: 1.0  patient no:  5 correct prediction rate: 1.0
t: 18  original: 1.0  patient no:  6 correct prediction rate: 1.0
t: 21  original: 1.0  patient no:  7 correct prediction rate: 1.0
t: 24  original: 1.0  patient no:  8 correct prediction rate: 0.6666666666666666
t: 27  original: 1.0  patient no:  9 correct prediction rate: 1.0
t: 30  original: 1.0  patient no:  10 correct prediction rate: 1.0
t: 33  original: 0.0  patient no:  11 correct prediction rate: 0.0
t: 36  original: 1.0  patient no:  12 correct prediction rate: 1.0
t: 39  original: 1.0  patient no:  13 correct prediction rate: 1.0
t: 42  original: 0.0  patient no:  14 correct 

t: 453  original: 1.0  patient no:  151 correct prediction rate: 1.0
t: 456  original: 1.0  patient no:  152 correct prediction rate: 1.0
t: 459  original: 0.0  patient no:  153 correct prediction rate: 0.0
t: 462  original: 1.0  patient no:  154 correct prediction rate: 1.0
t: 465  original: 1.0  patient no:  155 correct prediction rate: 1.0
t: 468  original: 0.0  patient no:  156 correct prediction rate: 1.0
t: 471  original: 0.0  patient no:  157 correct prediction rate: 0.6666666666666666
t: 474  original: 1.0  patient no:  158 correct prediction rate: 0.0
t: 477  original: 1.0  patient no:  159 correct prediction rate: 1.0
t: 480  original: 1.0  patient no:  160 correct prediction rate: 1.0
t: 483  original: 1.0  patient no:  161 correct prediction rate: 1.0
t: 486  original: 1.0  patient no:  162 correct prediction rate: 1.0
t: 489  original: 0.0  patient no:  163 correct prediction rate: 0.0
t: 492  original: 1.0  patient no:  164 correct prediction rate: 1.0
t: 495  original: 1

In [164]:
f1=0
f1=(tpc)/((tpc+0.5*(fpc+fnc)))
print('f1 score:', f1)

f1 score: 0.917098445595855


In [165]:
import math
mcc=0

mcc=(tpc*tnc-fpc*fnc)/math.sqrt((tpc+fpc)*(tpc+fnc)*(tnc+fpc)*(tnc+fnc))
print('mcc: ',mcc)
precision=tpc/(tpc+fpc)
print('precision: ', precision)
recall=tpc/(tpc+fnc)
print('recall: ',recall)
acc=(tpc+tnc)/(tpc+tnc+fpc+fnc)
print('accuracy: ', acc)

mcc:  0.6506657925275016
precision:  0.8939393939393939
recall:  0.9414893617021277
accuracy:  0.873015873015873


In [166]:
tpc,tnc, fpc, fnc

(177, 43, 21, 11)

In [161]:
import lime
import shap

In [154]:
#would have to do lime and shape later

In [155]:
print(Counter(trainy))

Counter({1.0: 562, 0.0: 187})


wrong_pred_indices=[]
d={}
svm_estimator=svm.SVC(C= 12, gamma= 0.001, kernel= 'poly',probability=True, random_state=10)
i=0

correct_ones_=[]#collects correct pd postive predictions
correct_zeros_=[]#collects correct pd negative predictions

wrong_ones_=[]#collect wrong pd positive predictions
wrong_zeros_=[]#collect wrong pd negative predictions

tp=[]# true positive
tn=[]# true negative
fp=[]# false positive
fn=[]# false negative

tridx=[]
tstidx=[]
acc=[]
mcc=[]
f1=[]
tsts=[]

for trids, tstids in kfold.split(x_train):
    
    trainx, testx= x_train[trids,:], x_train[tstids,:]
    trainy, testy= y_train[trids], y_train[tstids]
    
    #perform oversampling
    #over_samp=ADASYN(sampling_strategy='all')#BorderlineSMOTE()
    #trainx, trainy=over_samp.fit_resample(trainx, trainy)
    #print(Counter(trainy))

    ''''for i in range(0, len(trainx),3):
        #print(type(x_train[i:i+3,:]))
        if trainy[i]==0:
            a=np.vstack((a,trainx[i:i+3,:]))
            a=np.vstack((a,trainx[i:i+3,:]))
            a=np.vstack((a,trainx[i:i+3,:]))
    
            for j in trainy[i:(i+3)]:
                sampy.append(j)
                sampy.append(j)
                sampy.append(j)
        else:
            a=np.vstack((a,trainx[i:i+3,:]))
            for j in trainy[i:(i+3)]:
                sampy.append(j)
    a=a[3:,:]
    print(a.shape), print(np.array(sampy).shape)
    
    trainx=a
    trainy=np.array(sampy)'''
    

    #trainx, trainy=oversampling_smote(trainx,trainy, xs=2)
    over_samp=ADASYN(sampling_strategy='not minority', n_neighbors=3, )#
    over_samp2=BorderlineSMOTE(sampling_strategy='not majority', m_neighbors=15)
    trainx, trainy=over_samp.fit_resample(trainx, trainy)
    #trainx, trainy=over_samp2.fit_resample(trainx, trainy)
    
    svm_estimator=svm.SVC(C= 10.5, gamma= 0.00152, tol=1e-3, kernel= 'rbf',probability=True, random_state=10)
    
    svm_estimator.fit(trainx, trainy)
    
    preds=svm_estimator.predict(x_train[tstids,:])
    
    acc.append(accuracy_score(testy,svm_estimator.predict(testx)))
    #mcc.append(matthews_corrcoef(testy, svm_estimator.predict(testx)))
    f1.append(f1_score(testy, svm_estimator.predict(testx)))
    

    i=i+1
    print(i)
#print('acc:', mean(acc))
#print('f1:', mean(f1))
#print('mcc:', mean(mcc))
